This is to scrap lyrics from the website for mayday's lyrics.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
# URL of the main page
url = "https://mojim.com/cnh100012.htm"

# Send a GET request
response = requests.get(url)

# Parse the webpage
soup = BeautifulSoup(response.text, 'html.parser')

# Find all 'dd' elements with class 'hb2'
dd_elements = soup.find_all('dd', class_='hb2')

# Initialize empty lists to store the URLs, song names, and album names
urls = []
song_names = []
album_names = []

# Iterate over each 'dd' element
for dd in dd_elements:
    # Get the album name from the 'hc1' span
    album_name = dd.find('span', class_='hc1').get_text(strip=True)
    
    # Find all 'a' elements inside the 'dd' element
    a_elements = dd.find_all('a')
    
    # Iterate over each 'a' element
    for a in a_elements:
        # Get the 'href' attribute of the 'a' element, which is the URL
        song_url = a['href']
        
        # Add the base URL if the URL is relative
        if song_url.startswith('/'):
            song_url = "https://mojim.com" + song_url
        
        # Append the URL to the list
        urls.append(song_url)
        
        # Get the song name from the 'title' attribute and remove the " 歌词" part
        song_name = a.get('title').replace(' 歌词', '')
        
        # Append the song name and album name to their respective lists
        song_names.append(song_name)
        album_names.append(album_name)

# Print the URLs, song names, and album names
for url, song_name, album_name in zip(urls, song_names, album_names):
    print(f"URL: {url}, Song Name: {song_name}, Album Name: {album_name}")


URL: https://mojim.com/cn100012x85.htm, Song Name: 外人 五月天( MayDay ), Album Name: 外人
URL: https://mojim.com/cny100012x85x1.htm, Song Name: 外人(诺亚方舟 10周年进化), Album Name: 外人
URL: https://mojim.com/cn100012x83.htm, Song Name: 知足 五月天( MayDay ), Album Name: 知足
URL: https://mojim.com/cny100012x83x1.htm, Song Name: 知足(诺亚方舟10周年进化), Album Name: 知足
URL: https://mojim.com/cn100012x81.htm, Song Name: MAYDAY FLY TO 2023 五月天线上跨年演唱会 诺亚方舟十周年特别版 五月天( MayDay ), Album Name: MAYDAY FLY TO 2023 五月天线上跨年演唱会 诺亚方舟十周年特别版
URL: https://mojim.com/cny100012x81x1.htm, Song Name: 突然好想你, Album Name: MAYDAY FLY TO 2023 五月天线上跨年演唱会 诺亚方舟十周年特别版
URL: https://mojim.com/cny100012x81x2.htm, Song Name: 为你写下这首情歌, Album Name: MAYDAY FLY TO 2023 五月天线上跨年演唱会 诺亚方舟十周年特别版
URL: https://mojim.com/cny100012x81x3.htm, Song Name: 最重要的小事, Album Name: MAYDAY FLY TO 2023 五月天线上跨年演唱会 诺亚方舟十周年特别版
URL: https://mojim.com/cn100012x79.htm, Song Name: 我们都活在进化之中 五月天( MayDay ), Album Name: 我们都活在进化之中
URL: https://mojim.com/cny100012x79x1.htm, Song Name: 我们都

get the raw lyrics content. 

In [3]:

def get_lyrics(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    dl_element = soup.find('dl', id='fsZx1')
    lyrics = ''
    # Check if the 'dl' element was found
    if dl_element is not None:
        lyrics = dl_element.get_text(separator='\n')
    return lyrics


In [4]:
# Initialize an empty list to store the lyrics
lyrics_list = []


# For each URL in the list, get the lyrics.
for url in urls:
    lyrics = get_lyrics(url)
    lyrics_list.append(lyrics)

# Now you have a list of lyrics corresponding to each URL.


In [7]:
df = pd.DataFrame({
    'URL': urls,
    'Lyrics': lyrics_list,
    'Title': song_names,
    'Album Title': album_names
})

# Drop rows where 'Lyrics' is empty or null
df = df[df['Lyrics'].astype(bool)]

# Remove anything in parenthesis, including the parenthesis, from the 'Title' column
df['Title'] = df['Title'].str.replace(r'\(.*\)', '', regex=True).str.strip()

# Remove duplicated rows based on the 'Title' column and keep the first occurrence
df = df.drop_duplicates(subset='Title', keep='first')


# Assuming you have a DataFrame named df
df.to_csv('lyrics_mayday.csv', index=False)


Convert the df to json

In [9]:
import pandas as pd
import jieba

# Assuming df is your DataFrame
# df = pd.DataFrame({
#     'URL': urls,
#     'Title': song_names,
#     'Lyrics': lyrics_list
# })

def get_chunks(content, url, title):
    content_length = len(content)
    content_tokens = len(list(jieba.cut(content, cut_all=False)))
    chunk = {
        "essay_title": title,
        "essay_url": url,
        "essay_date": "NA",
        "essay_thanks": "NA",
        "content": content,
        "content_length": content_length,
        "content_tokens": content_tokens,
        "embedding": []
    }
    return [chunk]  # As chunks is an array, wrap the chunk object in a list

data = {
    "current_date": "2023-03-01",
    "author": "Your name here",
    "url": "Your URL here",
    "length": df['Lyrics'].apply(len).sum().item(),
    "tokens": df['Lyrics'].apply(lambda x: len(list(jieba.cut(x, cut_all=False)))).sum().item(),
    "essays": []
}

for idx, row in df.iterrows():
    essay = {
        "title": row['Title'],
        "url": row['URL'],
        "date": "NA",
        "thanks": "NA",
        "content": row['Lyrics'],
        "length": len(row['Lyrics']),
        "tokens": len(list(jieba.cut(row['Lyrics'], cut_all=False))),
        "chunks": get_chunks(row['Lyrics'], row['URL'], row['Title'])
    }
    data["essays"].append(essay)

# At this point, `data` is a Python dictionary that matches the PGJSON structure.
# You can convert it to a JSON string using json.dumps:
import json
json_str = json.dumps(data, ensure_ascii=False)

# Write the JSON string to a file named pg.json
with open('pg.json', 'w', encoding='utf8') as f:
    f.write(json_str)


Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/zf/xdqbk9tx0kx9s5z48tv58tf40000gn/T/jieba.cache
Loading model cost 0.413 seconds.
Prefix dict has been built successfully.
